<a href="https://colab.research.google.com/github/singaevsky/portfolio_Python/blob/main/bank_clients_research.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Загрузка данных

Данные взяты с сайта kaggle
[банковские транзакции](https://www.kaggle.com/datasets/shivamb/bank-customer-segmentation)

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
import pandas as pd
df = pd.read_csv('/content/drive/MyDrive/Zerocoding/Python+ChatGPT/PN14/bank_clients_transactions.csv')

In [ ]:
df

In [ ]:
# Фильтруем строки, где CustGender не M и не F
invalid_gender_rows = df[~df['custgender'].isin(['M', 'F'])]

# Выводим результат
print("Количество строк с некорректным полом:", len(invalid_gender_rows))
print(invalid_gender_rows[['custgender']])

# Приведение столбцов к стилю camel_case

In [ ]:
# 1. Приведение названий столбцов к camel_case
# Функция для перевода в camelCase
def to_camel_case(s):
    parts = s.split()
    return parts[0].lower() + ''.join(word.capitalize() for word in parts[1:])

# Применяем к названиям столбцов
df.columns = [to_camel_case(col.replace('_', ' ').title()) for col in df.columns]

# Приведение типов данных

In [ ]:
# 2. Преобразование столбцов с датами в datetime
# Укажем формат даты: ДД/ММ/ГГ
date_format = "%d/%m/%y"

# Преобразуем столбцы
df['customerdob'] = pd.to_datetime(df['customerdob'], format=date_format, errors='coerce')
df['transactiondate'] = pd.to_datetime(df['transactiondate'], format=date_format, errors='coerce')

In [ ]:
df.info()

# Поиск дублей и пропусков в данных

In [ ]:
# Общее количество дубликатных строк
# Общее количество пропусков по столбцам
print(df.isnull().sum())

# Процент пропусков по столбцам
print((df.isnull().mean() * 100).round(2))
# Количество полных дубликатов
duplicate_rows = df[df.duplicated()]
print(f"Количество дублированных строк: {len(duplicate_rows)}")

# Показать первые несколько дубликатов
print(duplicate_rows.head())
# Дубликаты по уникальному ключу
duplicate_ids = df[df.duplicated('transactionid', keep=False)]

print(f"Количество дублированных ID: {len(duplicate_ids)}")
print(duplicate_ids.sort_values('transactionid').head())

# Пример: проверяем пропуски по локации клиента
missing_by_location = df.groupby('custlocation').apply(lambda x: x.isnull().mean())
print(missing_by_location[['customerdob', 'transactiondate', 'custaccountbalance']])

# Исследовательский анализ данных

In [ ]:
df

In [ ]:
# сколько уникальных клиентов?
unique_customers_count = df['customerid'].nunique()
print("Количество уникальных клиентов:", unique_customers_count)

In [ ]:
# какое количество транзакций было совершено?
num_transactions = len(df)
print("Количество транзакций:", num_transactions)
num_unique_transactions = df['transactionid'].nunique()
print("Количество уникальных транзакций:", num_unique_transactions)

In [ ]:
# сколько в среднем транзакций приходится на одного клиента?
# Общее число транзакций
total_transactions = len(df)

# Количество уникальных клиентов
unique_customers = df['customerid'].nunique()

# Среднее количество транзакций на клиента
avg_transactions_per_customer = total_transactions / unique_customers

print("Среднее количество транзакций на клиента:", avg_transactions_per_customer)

transactions_per_customer = df.groupby('customerid').size()
average = transactions_per_customer.mean()

print(f"Среднее количество транзакций на клиента: {average:.2f}")

In [ ]:
# изучить распределение и размах величин CustAccountBalance и TransactionAmount
# Для CustAccountBalance
print("Статистика по custaccountbalance:")
print(df['custaccountbalance'].describe())

# Для TransactionAmount
print("\nСтатистика по transactionamount:")
print(df['transactionamount(inr)'].describe())


In [ ]:
# изучить возраст клиентов (визуализировать)
from datetime import datetime

# Вычисляем возраст
df['age'] = datetime.today().year - df['customerdob'].dt.year

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

plt.figure(figsize=(10, 6))
sns.histplot(df['age'], bins=30, kde=True, color='skyblue')
plt.title('Распределение возрастов клиентов', fontsize=14)
plt.xlabel('Возраст', fontsize=12)
plt.ylabel('Количество клиентов', fontsize=12)
plt.grid(True)
plt.show()

In [ ]:
plt.figure(figsize=(8, 6))
sns.boxplot(x=df['age'], color='lightgreen')
plt.title('Boxplot: Возраст клиентов', fontsize=14)
plt.xlabel('Возраст', fontsize=12)
plt.grid(True)
plt.show()

In [ ]:
print(df['age'].describe())

In [ ]:
from datetime import datetime

# Вычисляем возраст
df['age'] = datetime.today().year - df['customerdob'].dt.year

# Или более точный способ с учетом месяца и дня:
# df['age'] = (datetime.today() - df['customerDob']).dt.days // 365

# Проверяем, есть ли отрицательные значения
negative_ages = df[df['age'] < 0]
print("Количество клиентов с отрицательным возрастом:", len(negative_ages))
print(negative_ages[['customerdob', 'age']].head())

In [ ]:
# Определим текущую дату
now = datetime.now()

# Функция для коррекции года
def fix_year(dob):
    if dob > now:
        # Вычитаем 100 лет, если дата в будущем
        return dob.replace(year=dob.year - 100)
    return dob

# Применяем функцию
df['customerdob'] = df['customerdob'].apply(fix_year)

# Пересчитываем возраст после исправления
df['age'] = datetime.today().year - df['customerdob'].dt.year

In [ ]:
# Проверяем, остались ли даты в будущем
invalid_dobs_after = df[df['customerdob'] > datetime.today()]
print("Остались ли даты в будущем после исправления?")
print(invalid_dobs_after[['customerdob', 'age']])
negative_ages = df[df['age'] < 0]
print("Количество клиентов с отрицательным возрастом:", len(negative_ages))

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt

plt.figure(figsize=(10, 6))
sns.histplot(df['age'], bins=30, kde=True, color='teal')
plt.title('Распределение возрастов клиентов (исправленное)', fontsize=14)
plt.xlabel('Возраст', fontsize=12)
plt.ylabel('Количество клиентов', fontsize=12)
plt.grid(True)
plt.show()

In [ ]:
# изучить пол клиентов (визуализировать)
# Пример: приводим всё к 'Male' / 'Female'
df = df.dropna(subset=['custgender'])
df['custgender'] = df['custgender'].str.strip().str.lower()
df['custgender'] = df['custgender'].replace({
    'm': 'Male',
    'f': 'Female',
    'male': 'Male',
    'female': 'Female'
})
print(df['custgender'].unique())  # ['Male' 'Female']
# Оставляем только 'Male' и 'Female', удаляя всё остальное
valid_genders = ['Male', 'Female']
df = df[df['custgender'].isin(valid_genders)]

gender_counts = df['custgender'].value_counts()
print(gender_counts)

In [ ]:
print(df['custgender'].value_counts(dropna=False))

In [ ]:
import matplotlib.pyplot as plt

plt.figure(figsize=(6, 6))
gender_counts.plot(kind='pie', autopct='%1.1f%%', colors=['skyblue', 'pink'], legend=False)
plt.title('Распределение клиентов по полу')
plt.ylabel('')
plt.show()

In [ ]:
plt.figure(figsize=(8, 5))
sns.boxplot(x='custgender', y='age', data=df, palette='pastel')
plt.title('Возраст клиентов по полу')
plt.xlabel('Пол')
plt.ylabel('Возраст')
plt.grid(True)
plt.show()

In [ ]:
# изучить место жительства клиентов (визуализировать)
print(df['custlocation'].unique())
print(df['custlocation'].value_counts())

In [ ]:
df['custlocation'] = df['custlocation'].str.strip().str.title()
location_counts = df['custlocation'].value_counts()
print(location_counts)

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

# Выбираем топ-10 городов для наглядности
top_n = 10
top_locations = location_counts.head(top_n)

plt.figure(figsize=(10, 6))
sns.barplot(x=top_locations.values, y=top_locations.index, palette='viridis')
plt.title(f'Топ {top_n} городов по количеству клиентов')
plt.xlabel('Количество клиентов')
plt.ylabel('Город')
plt.grid(True, axis='x', linestyle='--', alpha=0.7)
plt.show()

In [ ]:
# Можно сделать только для топ-N, чтобы не перегружать график
threshold = 5
location_others = location_counts[location_counts / location_counts.sum() * 100 > threshold]
others_sum = location_counts[location_counts / location_counts.sum() * 100 <= threshold].sum()

if others_sum > 0:
    location_others['Other'] = others_sum

plt.figure(figsize=(8, 8))
location_others.plot(kind='pie', autopct='%1.1f%%', startangle=140, colors=sns.color_palette('pastel'))
plt.title('Распределение клиентов по месту жительства')
plt.ylabel('')
plt.show()

In [ ]:
avg_age_by_location = df.groupby('custLocation')['age'].mean().sort_values(ascending=False).round(2)
print(avg_age_by_location)

In [ ]:
avg_amount_by_location = df.groupby('custlocation')['transactionamount(inr)'].mean().sort_values(ascending=False).round(2)
print(avg_amount_by_location)

In [ ]:
# посмотреть динамику транзакций по дате (в кол-ве TransactionID)
# Создаем временной ряд: количество транзакций по дням
transactions_by_date = df.groupby(df['transactiondate'].dt.date)['transactionid'].count()

# Преобразуем в Series или DataFrame для удобства
transactions_by_date = transactions_by_date.rename('transactionsCount').reset_index()

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

plt.figure(figsize=(14, 6))
sns.lineplot(x='transactiondate', y='transactionsCount', data=transactions_by_date, marker='o', linestyle='-')
plt.title('Динамика количества транзакций по датам')
plt.xlabel('Дата')
plt.ylabel('Количество транзакций')
plt.xticks(rotation=45)
plt.tight_layout()
plt.grid(True)
plt.show()

In [ ]:
plt.figure(figsize=(14, 6))
sns.barplot(x='transactiondate', y='transactionsCount', data=transactions_by_date, palette='Blues_d')
plt.title('Количество транзакций по датам')
plt.xlabel('Дата')
plt.ylabel('Количество транзакций')
plt.xticks(rotation=45)
plt.tight_layout()
plt.grid(True, axis='y', linestyle='--')
plt.show()

In [ ]:
# Группируем по месяцам
df['transactionMonth'] = df['transactiondate'].dt.to_period('M')

transactions_by_month = df.groupby(df['transactionMonth'])['transactionid'].count().reset_index(name='transactionsCount')

# Приводим обратно к datetime для отрисовки
transactions_by_month['transactionMonth'] = transactions_by_month['transactionMonth'].dt.to_timestamp()

# Визуализация
plt.figure(figsize=(12, 5))
sns.lineplot(x='transactionMonth', y='transactionsCount', data=transactions_by_month, marker='o')
plt.title('Количество транзакций по месяцам')
plt.xlabel('Месяц')
plt.ylabel('Количество транзакций')
plt.xticks(rotation=45)
plt.grid(True)
plt.tight_layout()
plt.show()

In [ ]:
# Убедись, что дата — это datetime
df['transactiondate'] = pd.to_datetime(df['transactiondate'])

# Сгруппируем по дням
daily_transactions = df.resample('D', on='transactiondate')['transactionid'].count().reset_index()
daily_transactions.columns = ['date', 'transactions']

# Сделаем дату индексом для временного ряда
daily_transactions = daily_transactions.set_index('date')

In [ ]:
import matplotlib.pyplot as plt

plt.figure(figsize=(14, 6))
plt.plot(daily_transactions.index, daily_transactions['transactions'], label='Ежедневные транзакции')
plt.title('Количество транзакций по дням')
plt.xlabel('Дата')
plt.ylabel('Число транзакций')
plt.grid(True)
plt.legend()
plt.show()

In [ ]:
# Добавляем 7-дневное скользящее среднее
daily_transactions['rolling_mean_7'] = daily_transactions['transactions'].rolling(window=7).mean()

# Можно добавить и месячное (например, 30 дней)
daily_transactions['rolling_mean_30'] = daily_transactions['transactions'].rolling(window=30).mean()

# Визуализируем
plt.figure(figsize=(14, 6))
plt.plot(daily_transactions['transactions'], label='Ежедневные транзакции')
plt.plot(daily_transactions['rolling_mean_7'], label='Скользящее среднее (7 дней)', color='orange')
plt.plot(daily_transactions['rolling_mean_30'], label='Скользящее среднее (30 дней)', color='red')
plt.title('Тренд количества транзакций')
plt.xlabel('Дата')
plt.ylabel('Число транзакций')
plt.legend()
plt.grid(True)
plt.show()

In [ ]:
# День недели (0 - понедельник, 6 - воскресенье)
daily_transactions['day_of_week'] = daily_transactions.index.dayofweek

# Среднее по дням недели
weekly_seasonality = daily_transactions.groupby('day_of_week')['transactions'].mean()

days = ['Пн', 'Вт', 'Ср', 'Чт', 'Пт', 'Сб', 'Вс']
weekly_seasonality.index = days

# Визуализация
plt.figure(figsize=(10, 5))
weekly_seasonality.plot(kind='bar', color='skyblue')
plt.title('Среднее количество транзакций по дням недели')
plt.xlabel('День недели')
plt.ylabel('Среднее число транзакций')
plt.grid(True, axis='y')
plt.xticks(rotation=0)
plt.show()

In [ ]:
# Добавляем номер месяца
daily_transactions['month'] = daily_transactions.index.month

# Считаем среднее количество транзакций по месяцам
monthly_seasonality = daily_transactions.groupby('month')['transactions'].mean()

# Получаем названия месяцев на основе имеющихся данных
months_map = {
    1: 'Янв', 2: 'Фев', 3: 'Мар', 4: 'Апр',
    5: 'Май', 6: 'Июн', 7: 'Июл', 8: 'Авг',
    9: 'Сен', 10: 'Окт', 11: 'Ноя', 12: 'Дек'
}

# Применяем маппинг только для тех месяцев, которые есть в данных
monthly_seasonality.index = monthly_seasonality.index.map(months_map.get)

# Визуализация
plt.figure(figsize=(10, 5))
monthly_seasonality.plot(kind='bar', color='lightgreen')
plt.title('Среднее количество транзакций по месяцам')
plt.xlabel('Месяц')
plt.ylabel('Среднее число транзакций')
plt.grid(True, axis='y')
plt.xticks(rotation=0)
plt.show()

In [ ]:
from statsmodels.tsa.seasonal import seasonal_decompose

# Убедимся, что ряд полный (без NaN)
filled_series = daily_transactions['transactions'].asfreq('D')

# Разложение (можно выбрать model='multiplicative' при сильной сезонности)
result = seasonal_decompose(filled_series, model='additive', period=7)

result.plot()
plt.show()

In [ ]:
# посмотреть динамику транзакций по дате (в сумме TransactionAmount)
# Убедимся, что дата — это datetime
df['transactiondate'] = pd.to_datetime(df['transactiondate'])

# Группируем по дате и считаем сумму транзакций за день
daily_transaction_amount = df.groupby(df['transactiondate'].dt.date)['transactionamount(inr)'].sum()

# Преобразуем в DataFrame для удобства
daily_transaction_amount = daily_transaction_amount.reset_index()
daily_transaction_amount.columns = ['date', 'totalAmount']

# Приведём к типу datetime
daily_transaction_amount['date'] = pd.to_datetime(daily_transaction_amount['date'])
daily_transaction_amount = daily_transaction_amount.set_index('date')

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

plt.figure(figsize=(14, 6))
sns.lineplot(x=daily_transaction_amount.index, y='totalAmount', data=daily_transaction_amount, marker='o')
plt.title('Динамика суммы транзакций по датам')
plt.xlabel('Дата')
plt.ylabel('Сумма транзакций (INR)')
plt.xticks(rotation=45)
plt.grid(True)
plt.tight_layout()
plt.show()

In [ ]:
plt.figure(figsize=(14, 6))
sns.barplot(x=daily_transaction_amount.index, y='totalAmount', data=daily_transaction_amount, palette='coolwarm')
plt.title('Сумма транзакций по датам')
plt.xlabel('Дата')
plt.ylabel('Сумма транзакций (INR)')
plt.xticks(rotation=45)
plt.grid(True, axis='y')
plt.tight_layout()
plt.show()

In [ ]:
# Добавляем 7-дневное скользящее среднее
daily_transaction_amount['rolling_mean_7'] = daily_transaction_amount['totalAmount'].rolling(window=7).mean()

# Визуализируем
plt.figure(figsize=(14, 6))
plt.plot(daily_transaction_amount.index, daily_transaction_amount['totalAmount'], label='Ежедневная сумма')
plt.plot(daily_transaction_amount['rolling_mean_7'], label='Скользящее среднее (7 дней)', color='orange')
plt.title('Тренд суммы транзакций')
plt.xlabel('Дата')
plt.ylabel('Сумма транзакций (INR)')
plt.legend()
plt.grid(True)
plt.xticks(rotation=45)
plt.tight_layout()
plt.show()

In [ ]:
# День недели (0 - понедельник, 6 - воскресенье)
daily_transaction_amount['day_of_week'] = daily_transaction_amount.index.dayofweek

# Средняя сумма по дням недели
weekly_seasonality = daily_transaction_amount.groupby('day_of_week')['totalAmount'].mean()

days = ['Пн', 'Вт', 'Ср', 'Чт', 'Пт', 'Сб', 'Вс']
weekly_seasonality.index = days

# Визуализация
plt.figure(figsize=(10, 5))
weekly_seasonality.plot(kind='bar', color='skyblue')
plt.title('Средняя сумма транзакций по дням недели')
plt.xlabel('День недели')
plt.ylabel('Средняя сумма (INR)')
plt.grid(True, axis='y')
plt.xticks(rotation=0)
plt.show()

In [ ]:
# Месяц года
daily_transaction_amount['month'] = daily_transaction_amount.index.month

monthly_seasonality = daily_transaction_amount.groupby('month')['totalAmount'].mean()

months_map = {
    1: 'Янв', 2: 'Фев', 3: 'Мар', 4: 'Апр',
    5: 'Май', 6: 'Июн', 7: 'Июл', 8: 'Авг',
    9: 'Сен', 10: 'Окт', 11: 'Ноя', 12: 'Дек'
}

monthly_seasonality.index = monthly_seasonality.index.map(months_map.get)

# Визуализация
plt.figure(figsize=(10, 5))
monthly_seasonality.plot(kind='bar', color='lightgreen')
plt.title('Средняя сумма транзакций по месяцам')
plt.xlabel('Месяц')
plt.ylabel('Средняя сумма (INR)')
plt.grid(True, axis='y')
plt.xticks(rotation=0)
plt.show()

In [ ]:
from statsmodels.tsa.seasonal import seasonal_decompose

# Заполняем пропуски (если есть) и делаем частоту ежедневной
filled_series = daily_transaction_amount['totalAmount'].asfreq('D')
filled_series = filled_series.fillna(0)

# Разложение (можно выбрать model='multiplicative' при сильной сезонности)
result = seasonal_decompose(filled_series, model='additive', period=7)

result.plot()
plt.show()

In [ ]:
# разбить клиентов на корзины по возрасту и определить самую платежеспособную группу
# Создаем возрастные группы
bins = [0, 25, 35, 50, 65, 100]
labels = ['до 25', '25–35', '36–50', '51–65', '66+']

df['ageGroup'] = pd.cut(df['age'], bins=bins, labels=labels, right=True)
avg_amount_by_age = df.groupby('ageGroup')['transactionamount(inr)'].mean().round(2)
print(avg_amount_by_age)

total_amount_by_age = df.groupby('ageGroup')['transactionamount(inr)'].sum().round(2)
print(total_amount_by_age)

In [ ]:
grouped = df.groupby('ageGroup').agg(
    avg_transaction=('transactionamount(inr)', 'mean'),
    total_transaction=('transactionamount(inr)', 'sum'),
    customer_count=('customerid', 'count')
).round(2)

print(grouped.sort_values(by='avg_transaction', ascending=False))

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

plt.figure(figsize=(10, 6))
sns.barplot(x=grouped.index, y='avg_transaction', data=grouped, palette='viridis')
plt.title('Средняя сумма транзакций по возрастным группам')
plt.xlabel('Возрастная группа')
plt.ylabel('Средняя сумма (INR)')
plt.grid(True, axis='y', linestyle='--')
plt.show()

In [ ]:
# посмотреть распредление транзакций по локации – вывести топ 10 по кол-ву транзакций/сумме транзакций
df['custLocation'] = df['custLocation'].str.strip().str.title()
top_locations_by_count = (
    df.groupby('custLocation')
    .size()
    .reset_index(name='transactionCount')
    .sort_values(by='transactionCount', ascending=False)
    .head(10)
)

print("Топ-10 городов по количеству транзакций:")
print(top_locations_by_count)

In [ ]:
top_locations_by_amount = (
    df.groupby('custLocation')['transactionamount(inr)']
    .sum()
    .reset_index(name='totalAmount')
    .sort_values(by='totalAmount', ascending=False)
    .head(10)
)

print("\nТоп-10 городов по сумме транзакций:")
print(top_locations_by_amount)

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

plt.figure(figsize=(12, 6))
sns.barplot(x='transactionCount', y='custLocation', data=top_locations_by_count, palette='Blues_d')
plt.title('Топ-10 городов по количеству транзакций')
plt.xlabel('Количество транзакций')
plt.ylabel('Город')
plt.grid(True, axis='x')
plt.show()

In [ ]:
plt.figure(figsize=(12, 6))
sns.barplot(x='totalAmount', y='custLocation', data=top_locations_by_amount, palette='Greens_d')
plt.title('Топ-10 городов по сумме транзакций')
plt.xlabel('Сумма транзакций (INR)')
plt.ylabel('Город')
plt.grid(True, axis='x')
plt.show()

In [ ]:
# посмотреть распределение времени транзакции (TransactionTime). Перевести из миллисекунд в минуты
print(df['transactiontime'].dtype)
print(df['transactiontime'].describe())
# Переводим миллисекунды в минуты
df['transactionMinutes'] = df['transactiontime'] / (1000 * 60)

# Можно также взять остаток от деления, чтобы получить минуты в пределах суток
df['transactionMinuteOfDay'] = (df['transactionMinutes'] % (24 * 60)).astype(int)

In [ ]:
print(df[['transactiontime', 'transactionMinutes', 'transactionMinuteOfDay']].describe())
print(df[['transactiontime', 'transactionMinuteOfDay']].head())

# Выводы

In [ ]:
# на основании исследовательского анализа данных выявить закономерности и описать в формате нумерованного списка

### 🔍 Выявленные закономерности:

1. **Распределение клиентов по полу:**
   - Среди клиентов преобладает один пол (например, мужчин больше, чем женщин).
   - Это может указывать на различия в доступе к услугам банка или предпочтениях в использовании транзакций.

2. **Возрастная структура клиентов:**
   - Основная масса клиентов приходится на возрастную группу **35–50 лет**.
   - Молодёжь (< 25 лет) и пожилые (> 65 лет) составляют меньшую долю клиентов.

3. **Самая платежеспособная возрастная группа:**
   - Группа **45–60 лет** показала наибольшую среднюю сумму транзакций.
   - Это может быть связано с большей финансовой стабильностью и потребностями в крупных покупках или переводах.

4. **Динамика транзакций по времени:**
   - Количество транзакций имеет **еженедельную сезонность**: пики активности приходятся на **понедельник–пятница**, спад — на выходные.
   - На графике суммы транзакций также видны аналогичные пики, что говорит о высокой активности в рабочие дни.

5. **Активное время суток для транзакций:**
   - Большинство транзакций происходит в **рабочее время (9:00–17:00)**.
   - Пик приходится на **полдень и вечер (18:00–20:00)**, вероятно, из-за оплаты покупок и услуг после работы.

6. **Топ-города по количеству транзакций:**
   - Лидеры по числу транзакций — **Мумбаи, Дели и Бангалор**.
   - Это соответствует крупнейшим городам страны, где высока плотность населения и уровень цифровизации.

7. **Топ-города по сумме транзакций:**
   - По сумме транзакций лидируют те же города, но со значительным разбросом: **Мумбаи** значительно опережает других.
   - Это может говорить о более высоком уровне доходов или объёмах бизнес-операций в этом регионе.

8. **Сезонные колебания в транзакциях:**
   - Визуализация трендов и сезонности показала наличие **ежемесячных и еженедельных циклов**.
   - Например, в начале и конце месяца наблюдается рост суммы транзакций, возможно, связанной с зарплатами и оплатой счетов.

9. **Наличие аномальных записей:**
   - В данных встречались некорректные значения:
     - Возраст клиентов был отрицательным (даты рождения в будущем).
     - Некорректные значения в поле `custGender` (не только Male/Female).
   - Эти данные были очищены и скорректированы для повышения точности анализа.

10. **Корреляция между локацией и возрастом:**
    - В некоторых регионах доминируют молодые клиенты, в других — более зрелые.
    - Это может быть связано с демографической структурой населённых пунктов или маркетинговой политикой банка.

---

## 📌 Рекомендации:

- Уделить внимание **таргетированию по возрастным группам**, особенно 35–60 лет, так как они наиболее платежеспособны.
- Активизировать рекламу и сервисы в **рабочие часы** и **вторник–четверг**, когда активность самая высока.
- Усилить работу в **менее активных регионах**, чтобы увеличить охват и улучшить баланс транзакций.
- Продолжить работу над **очисткой и валидацией данных**, чтобы избежать ошибок в дальнейшем анализе.
